# Anytime heuristic search 

In [161]:
import copy
import math
from random import shuffle, randint
from heapq import heappop, heappush, heapify
import time

EPS = 0.00001

## Grid Search Domain Representation

In [162]:
class Map:
    '''
    Square grid map class represents the environment for our moving agent
        - width -- the number of columns in the grid
        - height -- the number of rows in the grid
        - cells -- the binary matrix, that represents the grid. 0 - cell is traversable, 1 - cell is blocked
    '''

    def __init__(self):
        '''
        Default constructor
        '''

        self.width = 0
        self.height = 0
        self.cells = []
    

    def ReadFromString(self, cellStr, width, height):
        '''
        Converting a string (with '#' representing obstacles and '.' representing free cells) to a grid
        '''
        self.width = width
        self.height = height
        self.cells = [[0 for _ in range(width)] for _ in range(height)]
        cellLines = cellStr.split("\n")
        i = 0
        j = 0
        for l in cellLines:
            if len(l) != 0:
                j = 0
                for c in l:
                    if c == '.':
                        self.cells[i][j] = 0
                    elif c == '#':
                        self.cells[i][j] = 1
                    else:
                        continue
                    j += 1
                if j != width:
                    raise Exception("Size Error. Map width = ", j, ", but must be", width )
                i += 1

        if i != height:
            raise Exception("Size Error. Map height = ", i, ", but must be", height )
    
    def SetGridCells(self, width, height, gridCells):
        '''
        Initialization of map by list of cells.
        '''
        self.width = width
        self.height = height
        self.cells = gridCells

    def InBounds(self, i, j):
        '''
        Check if the cell is on a grid.
        '''
        return (0 <= j < self.width) and (0 <= i < self.height)
    
    def Traversable(self, i, j):
        '''
        Check if the cell is not an obstacle.
        '''
        return not self.cells[i][j]


    def LegalMove(self, i1, j1, i2, j2):
      return self.InBounds(i1, j2) and self.Traversable(i1,j2) and self.InBounds(i2,j1) and self.Traversable(i2, j1)


    def GetNeighbours(self, i, j):
        '''
        Returns a list of neighbouring cells as (i,j) tuples. 
        Fucntions should returns such neighbours, that allows both cardinal and diagonal moves, 
        but dissalows cutting corners and squezzing. 

        Parameters:
            i, j (int, int): Position on the grid map, for which neighbours are needed

        Returns:
            neighbours(list[(int, int)]): List of neighbours grid map (i, j) coordinates.   
        '''
        neighbors = []
        di = [+1,+1,0,-1,-1,-1,0,+1]
        dj = [0,+1,+1,+1,0,-1,-1,-1]
        for k in range(8):
          x = i+di[k]
          y = j+dj[k]
          if self.InBounds(x, y) and self.Traversable(x, y) and self.LegalMove(i, j, x, y):
            neighbors.append((x,y))
       
#TODO (copy from previous labs)
        return neighbors

    def CalculateCost(self, i1, j1, i2, j2):
        '''                    
        Computes cost of simple moves between cells on grid, 
        or euclidean distance, if there isnt a simple move between cells

        Args:
            i1, j1 (int, int): Positions of first cell on the grid map
            i2, j2 (int, int): Positions of first cell on the grid map
   

        Returns:
            [float]: cost of simple moves or euclidean distance.   
        '''

        if(abs(i1 - i2) + abs(j1 - j2) == 2):
            return 1.41421356237
        elif (abs(i1 - i2) + abs(j1 - j2) == 1):
            return 1.0
        else:
            return math.sqrt(abs(i1 - i2) ** 2 + abs(j1 - j2) ** 2)


In [163]:
class Node:
    '''
    Class represents a grid search node

    - i, j: coordinates of corresponding grid element
    - g: g-value of the node
    - h: h-value of the node
    - F: f-value of the node
    - parent: pointer to the parent-node 
    '''

    def __init__(self, coord, g = math.inf, h = math.inf, F = None, parent = None):
        self.i = coord[0]
        self.j = coord[1]
        self.g = g
        self.h = h
        if F is None:
            self.F = self.g + h
        else:
            self.F = F        
        self.parent = parent
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j)
    
    def __lt__(self, other):
        return (self.F < other.F) or ((self.F == other.F) and (self.g < other.g))


In [164]:
def EuclidDistance(node1, node2):
    '''
    Returns a euclidean distance between cells corresponding to nodes
    '''
    return math.sqrt((node1.i - node2.i) ** 2 + (node1.j - node2.j) ** 2)


In [165]:
def MakePath(goal):
    '''
    Creates a path by tracing parent pointers from the goal node to the start node
    It also returns path's length.
    '''

    length = goal.g
    current = goal
    path = []
    while current.parent:
        path.append(current)
        current = current.parent
    path.append(current)
    return path[::-1], length


## Implementing OPEN and CLOSED

In [166]:
class Open:

    def __init__(self):
      self.heap = []
#TODO (copy from previous labs)

    def __iter__(self):
      return iter(self.heap)
#TODO (copy from previous labs)
    
    def __len__(self):
      return len(self.heap)
#TODO (copy from previous labs)

    def IsEmpty(self):
      if len(self.heap) != 0:
         return False
      return True
      '''
      IsEmpty should inform whether the OPEN is exhausted or not in the former case the search main loop should be interrupted
      '''
#TODO (copy from previous labs)

    def AddNode(self, item : Node, *args):
      heappush(self.heap,item)
      return
      '''
        AddNode is the method that puts (e.g. inserts or updates) the node to OPEN
        When implementing it do not forget to handle all possible cases:
         - node already in OPEN but the new g-value is better;
         - node already in OPEN but the new g-value is worse;
        - node is not in OPEN yet.
      '''
#TODO (copy from previous labs)

    def GetBestNode(self, *args):
      return heappop(self.heap)
      '''
        GetBestNode is the method that 
         i) finds the best node, i.e. the one with the lowest f-value (f=g+h) (for Dijkstra h=0),
         ii) removes it from OPEN and 
         iii) returns it
      '''
#TODO (copy from previous labs)
    
    def UniteAndUpdate(self, other, newWeight):
      for i in range(len(self.heap)):
        self.heap[i].F = self.heap[i].g + newWeight * self.heap[i].h
      heapify(self.heap)
      for element in other:
        element.F = element.g + newWeight * element.h
        heappush(self.heap,element)
      '''
        Unite Open set with other container and update F-values of nodes with new weight:
        
      f(Node) = g(Node) + weignt * h(Node)
      '''
#TODO


In [167]:
class Closed:
    
    def __init__(self):
        self.elements = {}
#TODO (copy from previous labs)

    def __iter__(self):
        return iter(self.elements)

#TODO (copy from previous labs)
    
    def __len__(self):
        len(self.elements)

#TODO (copy from previous labs)

    def AddNode(self, item : Node):
        key = item.i,item.j
        self.elements[key] = item 
        '''
        AddNode is the method that inserts the node to CLOSED
        '''
#TODO (copy from previous labs)

    def WasExpanded(self, item : Node):
        tmp = item.i,item.j
        if tmp in self.elements:
          return True
        else:
          return False
        '''
        WasExpanded is the method that checks if a node has been expanded
        '''
#TODO (copy from previous labs)

    def GetNode(self, i, j):
        key = i,j
        if key in self.elements : 
          return self.elements[key]
        else:
          return None
        '''
        Finds node from Closed set by (i, j) coordinates on grid and returns it. Returns None if node was not found.
        '''
#TODO (copy from previous labs)


## Weighted A*

In [168]:
def WAStar(startCoord, goalCoord, gridMap, hFunc, weight):
    '''
    Runs weighted A* search algorithm without re-expansion on any domain.

    Parameters:
        startCoord (tuple(int, int)): The start state of search in useful for your implementation form.
        goalCoord (tuple(int, int)): The goal state of search in useful for your implementation form.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        weight (float): The weight of heuristics in F-value computation.

    Returns:
        pathFound (bool): Path was found or not.   
        lastNode (Node): The last node in path. None if path was not found.
    '''

    OPEN = Open()
    CLOSED = Closed()

    GoalNode = Node(goalCoord)
    StartNode = Node(startCoord,0,0)
    StartNode.h = hFunc(StartNode,GoalNode)
    OPEN.AddNode(StartNode)
    while not OPEN.IsEmpty() :
      current = OPEN.GetBestNode()
      if CLOSED.WasExpanded(current):
        continue
      CLOSED.AddNode(current)
      if GoalNode == current :
          return True,current
      for neighbor in gridMap.GetNeighbours(current.i,current.j):
        state = Node((neighbor[0],neighbor[1]))
        state.parent = current
        state.g = current.g + gridMap.CalculateCost(current.i,current.j,state.i,state.j)
        state.h = hFunc(state,GoalNode)
        state.F = state.g + weight*state.h
        OPEN.AddNode(state)
#TODO (copy from previous labs)
    return False, None


## Naive Anytime A*

In [169]:
def NaiveATAStar(startCoord, goalCoord, gridMap, hFunc, startWeight = 3.0, stepWeight = 0.5):
    '''
    Repeatedly runs weighted A* search algorithm without re-expansion on any domain, 
    decreasing current weight from startWeight to 1.0 by stepWeight.

    Parameters:
        startCoord (tuple(int, int)): The start state of search in useful for your implementation form.
        goalCoord (tuple(int, int)): The goal state of search in useful for your implementation form.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        startWeigth (float): The initial weight of heuristics in F-value computation. Must be greater or equal to 1.0
        stepWeigth (float): The value by which the weight will be reduced, if it is provided by the algorithm 

    Returns:
        pathFound (bool): Path was found or not.   
        lastNode (Node): The last node in path. None if path was not found.
    '''

    weight = startWeight if (startWeight >= 1) else 1
    result = WAStar(startCoord, goalCoord, gridMap, hFunc, weight)
    yield result

    while True:
        result = WAStar(startCoord, goalCoord, gridMap, hFunc, weight)
        yield result
        if weight <= 1:
            break
        weight = (weight - stepWeight) if (weight - stepWeight) >= 1 else 1


## Anytime Repairing A*

In [170]:
def ComputeActualWeight(OPEN, INCONS, weight, goalNode):
    minOpen = math.inf
    minIncons = math.inf
    if not OPEN.IsEmpty():
        minOpen = min(v.g + v.h for v in OPEN)
    if not INCONS.IsEmpty():
        minIncons = min(v.g + v.h for v in INCONS)
    minG = min(minOpen, minIncons)

    return min(weight, goalNode.g / minG)

def ImprovePath(goalNode, gridMap, hFunc, OPEN, INCONS, CLOSED, weight):
#TODO
    '''
    Runs a modified version of weighted A* search algorithm without re-expansion on any domain.

    Parameters:
        goalNode: The goal state.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        weight (float): The weight of heuristics in F-value computation.

    Returns:
        pathFound (bool): Path was found or not.   
        lastNode (Node): The last node in path. None if path was not found.
    '''
    while len(OPEN.heap) > 0:
      #if goalNode.g + goalNode.h > OPEN.heap[0].F:
       # print("entered comparison")
      minOpen = OPEN.GetBestNode()
      CLOSED.AddNode(minOpen)
      if goalNode == minOpen :
          return True, minOpen
      for neighbor in gridMap.GetNeighbours(minOpen.i,minOpen.j):
    #    print(neighbor[0],neighbor[1])
        new_state = Node((neighbor[0],neighbor[1]))
        if not CLOSED.WasExpanded(new_state):
          new_state.g = math.inf
        else:
          new_state = CLOSED.GetNode(neighbor[0],neighbor[1])
        if new_state.g > minOpen.g + gridMap.CalculateCost(minOpen.i,minOpen.j,new_state.i,new_state.j):
          new_state.g = minOpen.g + gridMap.CalculateCost(minOpen.i,minOpen.j,new_state.i,new_state.j)
          new_state.h =  hFunc(new_state,goalNode)
          new_state.F = weight*new_state.h+ new_state.g
          new_state.parent = minOpen
          if not CLOSED.WasExpanded(new_state):
            OPEN.AddNode(new_state)
          else:
          #   CLOSED.elements[neighbor[0],neighbor[1]].g = new_state.g
            #new_state.g =new_state.F =0
            #oldNode = CLOSED.GetNode(neighbor[0],neighbor[1])
            #if oldNode.g>new_state.g:
            INCONS.AddNode(new_state)
            
      #else:
       # return True,OPEN.heap[0]
    return False, None
        
    

def ARAStar(startCoord, goalCoord, gridMap, hFunc, startWeight = 3.0, stepWeight = 0.5):
#TODO
    '''
    Runs Anytime Repairing A* search algorithm.

    Parameters:
        startCoord (tuple(int, int)): The start state of search in useful for your implementation form.
        goalCoord (tuple(int, int)): The goal state of search in useful for your implementation form.
        gridMap: An additional domain information (such as grid map).
        hFunc (function): The heuristics function for currently using domain.
        startWeigth (float): The initial weight of heuristics in F-value computation. Must be greater or equal to 1.0
        stepWeigth (float): The value by which the weight will be reduced, if it is provided by the algorithm  (not used)

    Returns:
        pathFound (bool): Path was found or not.   
        lastNode (Node): The last node in path. None if path was not found.
    '''
    OPEN = Open()
    INCONS = Open()
    CLOSED = Closed()

    StartNode = Node(startCoord,0,0)
    GoalNode = Node(goalCoord)
    h = hFunc(StartNode,GoalNode)
    StartNode.h = h
    StartNode.F = startWeight*h
    OPEN.AddNode(StartNode)
    weight = startWeight
    PathFound,GoalNode = ImprovePath(GoalNode, gridMap, hFunc, OPEN, INCONS, CLOSED, weight)
    
    yield PathFound,GoalNode
    ActualWeight = ComputeActualWeight(OPEN, INCONS, weight, GoalNode)
    while ActualWeight > 1 :
      weight = min(ActualWeight, weight - stepWeight)
      weight = max(weight, 1.0)
      OPEN.UniteAndUpdate(INCONS,weight)
      CLOSED = Closed()
      INCONS = Open()
      PathFound,GoalNode = ImprovePath(GoalNode, gridMap, hFunc, OPEN, INCONS, CLOSED, weight)
      ActualWeight = ComputeActualWeight(OPEN, INCONS, weight, GoalNode)
      yield PathFound,GoalNode 


# Experiment

In [171]:
def SimpleTest(SearchGenerator, task, heuristicFunction, startWeigth, stepWeigth, maxTime):
    '''
    SimpleTest runs SearchGenerator on one task (use a number from 0 to 25 to 
    choose a certain debug task on simple map or None to choose a random task 
    from this pool) with *args as optional arguments and displays:
     - 'Path found!' and some statistics -- path was found
     - 'Path not found!' -- path was not found
     - 'Execution error' -- an error occurred while executing the SearchFunction In first two cases function also draws visualisation of the task

    Parameters:
        SearchGenerator (generator):
        task (int):
        heuristicFunction (function): The heuristics function for currently using domain.
        startWeigth (float): The initial weight of heuristics in F-value computation. Must be greater or equal to 1.0
        stepWeigth (float): The value by which the weight will be reduced, if it is provided by the algorithm 
        maxTime (float): The amount of time given for the pathfinding procedure. Set in seconds.  

    Returns:
        pathFound (bool): Path was found or not.   
        length (Node): The length of the last found path.
        w (int): The sub-optimality value of the last found path.. 
    '''
    
    height = 15
    width = 30
    mapstr = '''
. . . # # . . . . . . . . # # . . . # . . # # . . . . . . .  
. . . # # # # # . . # . . # # . . . . . . # # . . . . . . . 
. . . . . . . # . . # . . # # . . . # . . # # . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # # . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # # . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # # # # # . . . . 
. . . # # . . # . . # . . # # . . . # . . # # # # # . . . . 
. . . . . . . # . . # . . # # . . . # . . # . . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # . . . . . . . . 
. . . # # . . # . . # . . # # . . . # . . # . . . . . . . . 
. . . # # . . . . . # . . . . . . . # . . . . . . . . . . . 
. . . # # # # # # # # # # # # # . # # . # # # # # # # . # # 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . .
'''

    taskMap = Map()
    taskMap.ReadFromString(mapstr, width, height)
    starts = [(0, 1), (6, 2), (5, 6), (13, 7), (4, 23)]
    goals = [(1, 28), (2, 29), (3, 20), (3, 20), (0, 0)]
    lengths = [47.8994949, 40.8994949, 38.2426407, 21.8284271, 47.4852814]

    if (task is None) or not (0 <= task < 5):
        task = randint(0, 4)

    start = Node(starts[task])
    goal = Node(goals[task])
    length = lengths[task]

    trueResult = WAStar((start.i, start.j), (goal.i, goal.j), taskMap, heuristicFunction, 1) 
    path = MakePath(trueResult[1])
    print("A* Path found! Length: {:.7f}".format(path[1]))

    start_time = time.time()
    lastResult = False, None

    iter_count = 0
    timer = 0.0
    finale_timer = 0.0
    w = 0.0
    for result in SearchGenerator((start.i, start.j), (goal.i, goal.j), taskMap, heuristicFunction, startWeigth, stepWeigth):
        iter_count += 1
        timer = time.time() - start_time
        
        if timer < maxTime:
            lastResult = result
            finale_timer = timer
            if lastResult[0]:
                wLength = lastResult[1].g
                w = wLength / length
                print(iter_count, "iter. Real sub-optimality value: {:.5f}".format(w))

                if w - 1 < EPS:
                    break
            else:
                print(iter_count, "iter. Weighted path not found!")
        else:
            break
    
    if lastResult[0]:
        wLength = lastResult[1].g
        w = wLength/length
        print("Weighted path found! Finale sub-optimality value: {:.7f}. Iterations: {:d}. Time: {:.7}".format(w, iter_count, finale_timer))
        return True, wLength, w
    else:
        print("Weighted path not found!")
        return False, math.inf, math.inf


In [172]:
for i in range(5):
    SimpleTest(NaiveATAStar, i, EuclidDistance, 5, 0.05, 0.05)

A* Path found! Length: 47.8994949
1 iter. Real sub-optimality value: 1.60692
2 iter. Real sub-optimality value: 1.60692
3 iter. Real sub-optimality value: 1.60692
4 iter. Real sub-optimality value: 1.60692
5 iter. Real sub-optimality value: 1.60692
6 iter. Real sub-optimality value: 1.60692
7 iter. Real sub-optimality value: 1.60692
Weighted path found! Finale sub-optimality value: 1.6069180. Iterations: 8. Time: 0.04520297
A* Path found! Length: 40.8994949
1 iter. Real sub-optimality value: 1.71079
2 iter. Real sub-optimality value: 1.71079
3 iter. Real sub-optimality value: 1.71079
4 iter. Real sub-optimality value: 1.71079
5 iter. Real sub-optimality value: 1.71079
6 iter. Real sub-optimality value: 1.71079
7 iter. Real sub-optimality value: 1.71079
Weighted path found! Finale sub-optimality value: 1.7107928. Iterations: 8. Time: 0.04481936
A* Path found! Length: 38.2426407
1 iter. Real sub-optimality value: 1.23085
2 iter. Real sub-optimality value: 1.23085
3 iter. Real sub-optimal

In [173]:
for i in range(5):
    SimpleTest(ARAStar, i, EuclidDistance, 5, 0.05, 0.5)

A* Path found! Length: 47.8994949
1 iter. Real sub-optimality value: 1.60692
2 iter. Real sub-optimality value: 1.63644
3 iter. Real sub-optimality value: 1.63644
4 iter. Real sub-optimality value: 1.62421
5 iter. Real sub-optimality value: 1.63644
6 iter. Real sub-optimality value: 1.63644
7 iter. Real sub-optimality value: 1.62421
8 iter. Real sub-optimality value: 1.01730
Weighted path found! Finale sub-optimality value: 1.0172951. Iterations: 9. Time: 0.4910896
A* Path found! Length: 40.8994949
1 iter. Real sub-optimality value: 1.71079
2 iter. Real sub-optimality value: 1.73105
3 iter. Real sub-optimality value: 1.74537
4 iter. Real sub-optimality value: 1.74537
5 iter. Real sub-optimality value: 1.07755
6 iter. Real sub-optimality value: 1.02026
7 iter. Real sub-optimality value: 1.02026
8 iter. Real sub-optimality value: 1.02026
9 iter. Real sub-optimality value: 1.00000
Weighted path found! Finale sub-optimality value: 1.0000000. Iterations: 9. Time: 0.2137365
A* Path found! Le